### Notebook to create Bedrock Agents and Action groups.

In [ ]:
import boto3, json, time

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()

In [ ]:
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

print(f"region:{region}")
print(f"account_id:{account_id}")

In [ ]:
# Agent foundation model name
agent_foundation_model = "anthropic.claude-3-5-sonnet-20240620-v1:0"

In [ ]:
# Create bedrock agent IAM policy
agent_bedrock_allow_policy_name = "ucai-iam-policy-name"

# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentStreamBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModelWithResponseStream",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}


bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [ ]:
print(agent_bedrock_policy)

In [ ]:
# Create IAM Role for the agent and attach IAM policies
agent_role_name = "ucai-iam-role-name"

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

In [ ]:
print(agent_role)

In [ ]:
# Create bedrock agent 

agent_name = "ucai-bra-name"
agent_description = "ucai-bra-description"
agent_instruction = "You are a weather agent to fetch the current weather in celsius and fahrenheit for a given location"

bedrock_agent_client = boto3.client('bedrock-agent')

create_agent_response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
print(create_agent_response)

In [ ]:
agent_id = create_agent_response['agent']['agentId']
print(f"agent_id:{agent_id}")

In [ ]:
# Prepare the function specifications
agent_functions = [

    {
        'name': 'get_weather_in_celsius',
        'description': 'Fetch the current weather in celsius for a given location and date',
        'parameters': {
            "location_id": {
                "description": "The unique identifier of the location to retrieve the temperature for",
                "required": True,
                "type": "string"
            },
            "fetch_date": {
                "description": "The specific date for which the temperature needs to be retrieved",
                "required": True,
                "type": "string"
            }
        },
        'requireConfirmation':'ENABLED'
    },
    {
        'name': 'get_weather_in_fahrenheit',
        'description': 'Fetch the current weather in fahrenheit for a given location and date',
        'parameters': {
            "location_id": {
                "description": "The unique identifier of the location to retrieve the temperature for",
                "required": True,
                "type": "string"
            },
            "fetch_date": {
                "description": "The specific date for which the temperature needs to be retrieved",
                "required": True,
                "type": "string"
            }
        },
        'requireConfirmation':'ENABLED'
    }
]

In [ ]:
# Prepare agent group using function schema
agent_action_group_name = "ucai-bda-action-group-name"
agent_action_group_description = "Actions to fetch the weather of a given location for a given date"

agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
print(response)

In [ ]:
# Create bedrock agent alias
agent_alias_name = "ucai-bra-alias-name"
agent_alias_description = "Alias for the weather agent"

create_alias_response = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name,
    description=agent_alias_description
)

print(create_alias_response)
agent_alias_id = create_alias_response['agentAlias']['agentAliasId']
print(f"agent_alias_id:{agent_alias_id}")

In [ ]:
print(agent_alias_id)